In [79]:
from pybaseball import salaries, statcast
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np

In [54]:
#import all pitches from 2021 into dataframe
statcast_data = statcast('2020-12-01', '2021-12-01')

This is a large query, it may take a moment to complete


/opt/anaconda3/envs/minimal_ds/lib/python3.10/site-packages/pybaseball/statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates


100%|██████████| 246/246 [01:03<00:00,  3.87it/s]


In [55]:
#view dataframe

statcast_data

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,...,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
112,FF,2021-11-02,93.7,1.39,6.72,"Smith, Will",493329,519293,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,X,6,ground_ball,0,2,2021,0.57,1.21,0.19,2.68,<NA>,<NA>,488726,2,9,Bot,98.12,136.43,<NA>,...,5,93.8,-28,93.4,2112,6.1,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.39,0.071,0.064,0.0,1,0,0,2,69,3,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,146,0.0,-0.136
117,FF,2021-11-02,92.9,1.38,6.72,"Smith, Will",493329,519293,NaN,foul,<NA>,<NA>,<NA>,<NA>,5,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,S,<NA>,NaN,0,1,2021,0.9,1.34,0.01,2.12,<NA>,<NA>,488726,2,9,Bot,<NA>,<NA>,<NA>,...,156,75.0,15,92.6,2206,6.3,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,69,2,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,145,0.0,-0.047
124,FF,2021-11-02,93.1,1.35,6.73,"Smith, Will",493329,519293,NaN,called_strike,<NA>,<NA>,<NA>,<NA>,6,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,S,<NA>,NaN,0,0,2021,0.81,1.52,0.78,2.13,<NA>,<NA>,488726,2,9,Bot,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,92.5,2216,6.2,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,69,1,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,143,0.0,-0.041
133,FF,2021-11-02,94.6,1.31,6.73,"Smith, Will",670541,519293,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,5,Yordan Alvarez flies out to left fielder Eddie...,W,L,L,HOU,ATL,X,7,fly_ball,3,2,2021,0.85,1.27,-0.23,2.66,<NA>,<NA>,488726,1,9,Bot,68.5,86.79,<NA>,...,312,92.7,39,93.8,2263,6.3,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.25,0.026,0.046,0.0,1,0,0,3,68,6,4-Seam Fastball,0,7,0,7,7,0,0,7,Infield shift,Standard,140,-0.001,-0.384
139,FF,2021-11-02,93.6,1.31,6.8,"Smith, Will",670541,519293,NaN,ball,<NA>,<NA>,<NA>,<NA>,13,Yordan Alvarez flies out to left fielder Eddie...,W,L,L,HOU,ATL,B,<NA>,NaN,2,2,2021,0.9,1.43,-1.15,1.51,<NA>,<NA>,488726,1,9,Bot,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,92.8,2239,6.2,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,68,5,4-Seam Fastball,0,7,0,7,7,0,0,7,Infield shift,Standard,152,0.0,0.117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3547,FF,2021-04-01,95.9,-1.92,6.15,"Márquez, Germán",605141,608566,NaN,called_strike,<NA>,<NA>,<NA>,<NA>,2,Mookie Betts singles on a fly ball to left fie...,R,R,R,COL,LAD,S,<NA>,NaN,3,1,2021,-0.17,0.86,0.07,3.09,<NA>,<NA>,<NA>,0,1,Top,<NA>,<NA>,<NA>,...,<NA>,<NA>,<NA>,95.4,2113,5.4,634615,608566,553869,543068,572008,658069,596115,606132,641658,453568,55.05,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,5,4-Seam Fastball,0,0,0,0,0,0

In [66]:
#identify which columns have either no unique values or one unique value - these will not be helpful in machine learning
unique = statcast_data.nunique()

In [71]:
#get index of columns with only one unique value
one_unique = statcast_data.nunique()[statcast_data.nunique() == 1].index

In [76]:
#get index of columns with no unique value (empty)
no_unique = statcast_data.nunique()[statcast_data.nunique() == 0].index 
no_unique

Index(['spin_dir', 'spin_rate_deprecated', 'break_angle_deprecated',
       'break_length_deprecated', 'tfs_deprecated', 'tfs_zulu_deprecated',
       'umpire', 'sv_id'],
      dtype='object')

In [83]:
statcast_data_drop_one_unique = statcast_data.drop(columns=one_unique)
statcast_data_drop_empty = statcast_data_drop_one_unique.drop(columns=no_unique)
statcast_data_drop_empty

,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,...,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp
112,FF,2021-11-02,93.7,1.39,6.72,"Smith, Will",493329,519293,field_out,hit_into_play,5,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,X,6,ground_ball,0,2,0.57,1.21,0.19,2.68,<NA>,<NA>,488726,2,9,Bot,98.12,136.43,518595,-4.348927,-136.263269,-7.335202,8.101619,29.384843,...,5,93.8,-28,93.4,2112,6.1,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.39,0.071,0.064,0.0,1,0,0,2,69,3,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,146,0.0,-0.136
117,FF,2021-11-02,92.9,1.38,6.72,"Smith, Will",493329,519293,NaN,foul,5,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,S,<NA>,NaN,0,1,0.9,1.34,0.01,2.12,<NA>,<NA>,488726,2,9,Bot,<NA>,<NA>,518595,-5.455433,-134.989926,-8.881689,12.188852,30.69001,...,156,75.0,15,92.6,2206,6.3,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.22,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,69,2,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,145,0.0,-0.047
124,FF,2021-11-02,93.1,1.35,6.73,"Smith, Will",493329,519293,NaN,called_strike,6,"Yuli Gurriel grounds out, shortstop Dansby Swa...",W,R,L,HOU,ATL,S,<NA>,NaN,0,0,0.81,1.52,0.78,2.13,<NA>,<NA>,488726,2,9,Bot,<NA>,<NA>,518595,-3.230974,-135.201801,-9.255781,10.67848,31.699974,...,<NA>,<NA>,<NA>,92.5,2216,6.2,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.3,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,69,1,4-Seam Fastball,0,7,0,7,7,0,0,7,Standard,Standard,143,0.0,-0.041
133,FF,2021-11-02,94.6,1.31,6.73,"Smith, Will",670541,519293,field_out,hit_into_play,5,Yordan Alvarez flies out to left fielder Eddie...,W,L,L,HOU,ATL,X,7,fly_ball,3,2,0.85,1.27,-0.23,2.66,<NA>,<NA>,488726,1,9,Bot,68.5,86.79,518595,-5.901934,-137.422092,-7.652311,12.118242,35.102245,...,312,92.7,39,93.8,2263,6.3,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.25,0.026,0.046,0.0,1,0,0,3,68,6,4-Seam Fastball,0,7,0,7,7,0,0,7,Infield shift,Standard,140,-0.001,-0.384
139,FF,2021-11-02,93.6,1.31,6.8,"Smith, Will",670541,519293,NaN,ball,13,Yordan Alvarez flies out to left fielder Eddie...,W,L,L,HOU,ATL,B,<NA>,NaN,2,2,0.9,1.43,-1.15,1.51,<NA>,<NA>,488726,1,9,Bot,<NA>,<NA>,518595,-8.265487,-135.578023,-10.936295,12.932808,31.370147,...,<NA>,<NA>,<NA>,92.8,2239,6.2,660906,519293,518595,518692,645277,663586,621020,592696,628338,594807,54.28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,68,5,4-Seam Fastball,0,7,0,7,7,0,0,7,Infield shift,Standard,152,0.0,0.117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3547,FF,2021-04-01,95.9,-1.92,6.15,"Márquez, Germán",605141,608566,NaN,called_strike,2,Mookie Betts singles on a fly ball to left fie...,R,R,R,COL,LAD,S,<NA>,NaN,3,1,-0.17,0.86,0.07,3.09,<NA>,<NA>,<NA>,0,1,Top,<NA>,<NA>,553869,5.561971,-139.787362,-4.509214,-3.312501,24.833759,...,<NA>,<NA>,<NA>,95.4,2113,5.4,634615,608566,553869,543068,572008,658069,596

In [89]:
cols_to_keep = ['pitch_type','release_speed', 'pfx_x', 'pfx_z', 'plate_x', 'plate_z', 'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'release_spin_rate', 'pitch_name', 'spin_axis']

data = statcast_data[cols_to_keep]
data

,pitch_type,release_speed,pfx_x,pfx_z,plate_x,plate_z,vx0,vy0,vz0,ax,ay,az,release_spin_rate,pitch_name,spin_axis
112,FF,93.7,0.57,1.21,0.19,2.68,-4.348927,-136.263269,-7.335202,8.101619,29.384843,-15.743652,2112,4-Seam Fastball,146
117,FF,92.9,0.9,1.34,0.01,2.12,-5.455433,-134.989926,-8.881689,12.188852,30.69001,-14.009571,2206,4-Seam Fastball,145
124,FF,93.1,0.81,1.52,0.78,2.13,-3.230974,-135.201801,-9.255781,10.67848,31.699974,-11.7058,2216,4-Seam Fastball,143
133,FF,94.6,0.85,1.27,-0.23,2.66,-5.901934,-137.422092,-7.652311,12.118242,35.102245,-14.577857,2263,4-Seam Fastball,140
139,FF,93.6,0.9,1.43,-1.15,1.51,-8.265487,-135.578023,-10.936295,12.932808,31.370147,-12.168466,2239,4-Seam Fastball,152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3547,FF,95.9,-0.17,0.86,0.07,3.09,5.561971,-139.787362,-4.509214,-3.312501,24.833759,-20.112617,2113,4-Seam Fastball,212
3668,FF,95.4,-0.06,0.68,0.52,3.95,5.875569,-139.018112,-1.839219,-1.810341,24.856947,-23.06789,<NA>,4-Seam Fastball,<NA>
3837,FF,96.1,-0.42,0.86,-0.61,3.65,3.953292,-140.09228,-3.371124,-6.331036,25.767684,-20.177597,2063,4-Seam Fastball,208
3941,FF,95.6,-0.25,0.88,0.19,2.6,6.352185,-139.293012,-5.428763,-4.405496,25.441128,-19.766641,2988,4-Seam Fastball,195


In [90]:
data.isna().sum()

pitch_type           2636
release_speed        2090
pfx_x                2056
pfx_z                2056
plate_x              2056
plate_z              2056
vx0                  2056
vy0                  2056
vz0                  2056
ax                   2056
ay                   2056
az                   2056
release_spin_rate    4715
pitch_name           2636
spin_axis            4679
dtype: int64

In [91]:
data.nunique()

pitch_type               13
release_speed           583
pfx_x                   461
pfx_z                   430
plate_x                 795
plate_z                 942
vx0                  718580
vy0                  718581
vz0                  718580
ax                   718581
ay                   718581
az                   718581
release_spin_rate      3103
pitch_name               12
spin_axis               361
dtype: int64